In [ ]:
!pip install tqdm
!pip install node2vec
!pip install demoji

!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter16_tree_data.csv
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter16_label_data.csv
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter16_source_tweet_data.csv
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter16_vectors.npy
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter16_graph.graphml
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/embeddings2.pickle

!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter15_tree_data.csv
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter15_label_data.csv
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter15_source_tweet_data.csv
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter15_vectors.npy
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/twitter15_graph.graphml
!wget https://raw.githubusercontent.com/dianacotelin/fakenews/main/drive-download-20240527T175238Z-001/embeddings.pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0

In [ ]:
!pip install nltk

In [ ]:
import torch
from transformers import DebertaTokenizer, DebertaModel
# Initialize DeBERTa model
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
deberta_model = DebertaModel.from_pretrained('microsoft/deberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Twitter 16

In [ ]:
import pandas as pd

# Read data from CSV files
twitter16_tree_df = pd.read_csv('./twitter16_tree_data.csv')
twitter16_label_df = pd.read_csv('./twitter16_label_data.csv')
twitter16_source_tweet_df = pd.read_csv('./twitter16_source_tweet_data.csv')


In [ ]:
twitter16_source_tweet_df['content']

In [ ]:
import pandas as pd
import re
import string
import demoji
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def remove_punctuation(text):
    return re.sub(r'[^A-Za-z0-9\s]+', '', text)

def remove_url_word(text):
    return text.replace("URL", "")

def remove_emojis(text):
    return demoji.replace(text, '')

def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

def clean_text(text):
    text = remove_url_word(text)
    text = remove_punctuation(text)
    text = remove_emojis(text)
    text = remove_stop_words(text)
    text = lemmatize_text(text)
    return text

# Apply the cleaning function to the DataFrame
twitter16_source_tweet_df['cleaned_content'] = twitter16_source_tweet_df['content'].apply(clean_text)

In [ ]:
twitter16_source_tweet_df['cleaned_content']

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset


class TwitterDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=64):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded_text = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {key: val.squeeze() for key, val in encoded_text.items()}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
deberta_model.to(device)

texts = list(twitter16_source_tweet_df['content'])
dataset = TwitterDataset(texts, tokenizer)
batch_size = 16  # Choose an appropriate batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model_outputs = []
deberta_model.eval()
from tqdm import tqdm
for batch in tqdm(dataloader):
    batch = {key: val.to(device) for key, val in batch.items()}

    with torch.no_grad():
        outputs = deberta_model(**batch)
        model_outputs.append(outputs)

all_outputs = {
    key: torch.cat([getattr(output, key) for output in model_outputs], dim=0)
    for key in model_outputs[0].keys()
}

In [ ]:
print(all_outputs['last_hidden_state'].shape)

In [ ]:
import numpy as np
embs = all_outputs['last_hidden_state'].cpu()
np.save('./twitter_16_deb_new_clean', embs.numpy())

In [ ]:
twitter16_vectors = np.load('./twitter_16_deb_new_clean.npy')
twitter16_vectors.shape

# Twitter 15

In [ ]:
import pandas as pd

# Read data from CSV files
twitter15_tree_df = pd.read_csv('./twitter15_tree_data.csv')
twitter15_label_df = pd.read_csv('./twitter15_label_data.csv')
twitter15_source_tweet_df = pd.read_csv('./twitter15_source_tweet_data.csv')


In [ ]:
twitter15_source_tweet_df['content']

0       🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...
1       an open letter to trump voters from his top st...
2       "america is a nation of second chances" —@potu...
3       brandon marshall visits and offers advice, sup...
4       rip elly may clampett: so sad to learn #beverl...
                              ...                        
1485    .@potus just announced new reforms to address ...
1486                      “after school satan clubs”? URL
1487    breaking news: according to documents released...
1488                     ebola vaccines? URL #news #today
1489    concerned airport passenger suits up in homema...
Name: content, Length: 1490, dtype: object

In [ ]:
import pandas as pd
import re
import string
import demoji
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def remove_punctuation(text):
    return re.sub(r'[^A-Za-z0-9\s]+', '', text)

def remove_url_word(text):
    return text.replace("URL", "")

def remove_emojis(text):
    return demoji.replace(text, '')

def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

def clean_text(text):
    # text = remove_url_word(text)
    # text = remove_punctuation(text)
    # text = remove_emojis(text)
    # text = remove_stop_words(text)
    # text = lemmatize_text(text)
    return text

# Apply the cleaning function to the DataFrame
twitter15_source_tweet_df['cleaned_content'] = twitter15_source_tweet_df['content'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
pip install contractions


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.1 MB/s eta 0:00:00


In [ ]:
pip install pyspellchecker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import string
import demoji
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import contractions
from textblob import TextBlob

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def remove_punctuation(text):
    return re.sub(r'[^A-Za-z0-9\s]+', '', text)

def remove_url_word(text):
    return text.replace("URL", "")

def remove_emojis(text):
    return demoji.replace(text, '')

def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

def to_lowercase(text):
    return text.lower()

def remove_numbers(text):
    return re.sub(r'\d+', '', text)

def expand_contractions(text):
    return contractions.fix(text)

def remove_extra_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

def remove_html_tags(text):
    return BeautifulSoup(text, "html.parser").get_text()

def correct_spelling(text):
    return str(TextBlob(text).correct())

def clean_text(text):
    text = remove_html_tags(text)
    text = expand_contractions(text)
    text = to_lowercase(text)
    text = remove_numbers(text)
    text = remove_url_word(text)
    text = remove_punctuation(text)
    text = remove_emojis(text)
    text = remove_stop_words(text)
    text = correct_spelling(text)
    text = lemmatize_text(text)
    text = remove_extra_whitespace(text)
    return text

# Apply the cleaning function to the DataFrame
twitter15_source_tweet_df['cleaned_content'] = twitter15_source_tweet_df['content'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
<ipython-input-15-ab843e1b59e1>:51: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


In [ ]:
twitter15_source_tweet_df['cleaned_content']

0       ca ask grand wizard endorsed hillaryclinton ne...
1       open letter tramp voter top strategistturnedde...
2       america nation second chance pot new reform so...
3       brandon marshall visit offer advice support br...
4       rip belly may clamped sad learn beverlyhillbil...
                              ...                        
1485    pot announced new reform address verse solitar...
1486                               school satan club curl
1487    breaking news according document released pres...
1488                        ebook vaccine curl news today
1489    concerned airport passenger suit homemade hazm...
Name: cleaned_content, Length: 1490, dtype: object

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset


class TwitterDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=64):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded_text = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {key: val.squeeze() for key, val in encoded_text.items()}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
deberta_model.to(device)

texts = list(twitter15_source_tweet_df['content'])
dataset = TwitterDataset(texts, tokenizer)
batch_size = 16  # Choose an appropriate batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model_outputs = []
deberta_model.eval()
from tqdm import tqdm
for batch in tqdm(dataloader):
    batch = {key: val.to(device) for key, val in batch.items()}

    with torch.no_grad():
        outputs = deberta_model(**batch)
        model_outputs.append(outputs)

all_outputs = {
    key: torch.cat([getattr(output, key) for output in model_outputs], dim=0)
    for key in model_outputs[0].keys()
}

100%|██████████| 94/94 [06:30<00:00,  4.15s/it]


In [ ]:
print(all_outputs['last_hidden_state'].shape)

torch.Size([1490, 64, 768])


In [ ]:
import numpy as np
embs = all_outputs['last_hidden_state'].cpu()
np.save('./twitter_15_deb_agg_clean', embs.numpy())

In [ ]:
twitter15_vectors = np.load('./twitter_15_deb_agg_clean.npy')
twitter15_vectors.shape

(1490, 64, 768)

In [ ]:
!cp ./twitter_15_deb_agg_clean.npy "/content/drive/MyDrive/[Licenta] Maria-Diana COTELIN - Network-aware Fake News Mitigation/datas/embs"